# Flair Document Embeddings

### Dokumentation

https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_5_DOCUMENT_EMBEDDINGS.md

# Import

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence
from umap import UMAP

import pandas as pd
import plotly.express as px



## Beispiel für ein Document Embedding

In [ ]:
text = '''Biden befürchtet Einmarsch "in den nächsten Tagen"
            Stand: 18.02.2022 06:01 Uhr
            Die Gefahr eines russischen Einmarsches in die Ukraine schätzt US-Präsident Biden als "sehr hoch" ein - schon in den kommenden Tagen könne es dazu kommen. Die russische Regierung wies den Vize-US-Botschafter in Moskau aus.
            US-Präsident Joe Biden befürchtet trotz aller Beteuerungen aus Moskau einen russischen Einmarsch in die Ukraine in den kommenden Tagen. Biden sagte, die Gefahr einer Invasion sei "sehr hoch". Nach seiner Einschätzung könne es "in den nächsten paar Tagen" dazu kommen. Es gebe keine Pläne dafür, dass er mit dem russischen Präsidenten Wladimir Putin telefonieren werde, fügte er hinzu.
            Der Kreml erklärte laut der russischen Agentur RIA, Bidens Warnung verstärke die Spannungen noch. Der stellvertretende Außenminister Sergej Werschinin wies vor dem UN-Sicherheitsrat die Befürchtungen des Westens vor einem bevorstehenden Einmarsch erneut zurück. "Ich denke, wir haben genug darüber spekuliert", sagte er. Eine Invasion sei entgegen der Warnungen ausgeblieben. In Richtung der USA und ihrer westlichen Verbündeten sagte Werschinin: "Mein Rat an Sie ist, sich nicht in eine unangenehme Situation zu begeben."
            Biden will am heutigen Freitag mit Verbündeten über das weitere Vorgehen beraten. Themen der Telefonschalte am Nachmittag (Ortszeit) sollten unter anderem die Aufstockung der russischen Truppen an der Grenze zur Ukraine und weitere diplomatische Bemühungen sein, hieß es aus dem Weißen Haus. Neben Kanadas Premierminister Justin Trudeau sollen führende Politiker aus Deutschland, Frankreich, Großbritannien, Italien, Polen und Rumänien an dem Gespräch teilnehmen, teilte Trudeaus Büro am Donnerstagabend (Ortszeit) mit. Auch die Europäische Union und die NATO seien vertreten.
            US-Außenminister Antony Blinken will sich nächste Woche mit seinem russischen Kollegen Sergej Lawrow treffen - solange Russland nicht in der Ukraine einmarschiert. Blinken habe vorgeschlagen, sich mit Lawrow "nächste Woche in Europa zu treffen. Die Russen haben mit Terminvorschlägen für Ende nächster Woche geantwortet, was wir unter der Bedingung akzeptiert haben, dass es keine russische Invasion der Ukraine gibt", erklärte der Sprecher des Außenministeriums, Ned Price, am Abend.
            Zuvor hatte Blinken vor dem UN-Sicherheitsrat ebenfalls gewarnt, Russland bereite sich auf einen Angriff in den kommenden Tagen vor. Russlands Plan sei, dafür einen Vorwand zu schaffen. "Dies könnte ein gewaltsames Ereignis sein, das Russland gegen die Ukraine vorbringen wird, oder eine unerhörte Anschuldigung, die Russland gegen die ukrainische Regierung erheben wird", sagte er. Möglich wären ihm zufolge ein vermeintlicher Terroranschlag in Russland, die "erfundene Entdeckung eines Massengrabes" und Vorwürfe eines Völkermordes, ein inszenierter Drohnenangriff auf Zivilisten oder ein vorgetäuschter oder echter Angriff mit Chemiewaffen.
            Russische Medien würden bereits "falsche Alarme" verbreiten, so Blinken. Ein russischer Angriff könne auch die ukrainische Hauptstadt Kiew einbeziehen. Der US-Außenminister betonte, Diplomatie sei weiter der wichtigste Weg zur Lösung der Krise. Er habe daher seinem russischen Amtskollegen Sergej Lawrow ein persönliches Treffen in der kommenden Woche vorgeschlagen. 
            '''

In [ ]:
# Erstellen eines Beispielsatzes
sentence = Sentence(text)

In [ ]:
# Initialisieren der Word Embeddings
embedding = WordEmbeddings('de')

# Initialisieren der Document Embeddings, mode = mean
document_embeddings = DocumentPoolEmbeddings([embedding])

In [ ]:
# Das Sentence-Objekt in das Documente Embedding einbetten
document_embeddings.embed(sentence)

# Check
print(sentence.embedding)
print(sentence.embedding.cpu().detach().numpy()) # als numpy array konvertiert

## Document Embeddings für die Bundestagsreden

In [ ]:
df = pd.read_xml('../data/bundesregierung.xml')

In [ ]:
# change dtype to datetime
df.loc[:, 'datum'] = pd.to_datetime(df.loc[:, 'datum'])

In [ ]:
# df = df.head(10)

In [ ]:
df.info()

In [ ]:
def create_numpy_array(text, document_embeddings):
    '''
    '''
    
    sentence = Sentence(text)
    document_embeddings.embed(sentence)
    
    return sentence.embedding.cpu().detach().numpy()  

In [ ]:
%%time

df.loc[:, 'doc_embedding'] = df.loc[:, 'rohtext'].apply(lambda text: create_numpy_array(text, document_embeddings))

In [ ]:
df.head()

In [ ]:
df.to_json('../data/reden-bundestag-doc-embeddings.json')

## Dimensionsreduktion

In [ ]:
reducer_3d = UMAP(n_components=3, metric='cosine', n_neighbors=15, min_dist=0.1)

In [ ]:
reduced_matrix_3d = reducer_3d.fit_transform(df.loc[:, 'doc_embedding'].to_list())

In [ ]:
reduced_matrix_3d.shape

In [ ]:
reduced_matrix_3d

In [ ]:
df_3d = pd.DataFrame.from_records(reduced_matrix_3d, columns=['x', 'y', 'z'])

In [ ]:
df_3d.head()

In [ ]:
df_all = df.join(df_3d)

In [ ]:
df_all.head()

## Visualisieren der Document Embeddings

In [ ]:
df_all.loc[:, 'Redner:innen'] = ''

In [ ]:
search_terms = [ 'Angela Merkel', 'Gerhard Schröder', 'Helmut Kohl', 'Bernd Neumann', 'Michael Naumann']

for search_term in search_terms:
    
    subframe = df_all.query(f"person == '{search_term}'")
    df_all.loc[subframe.index, 'Redner:innen'] = search_term

In [ ]:
params = {'hover_data': {c: False for c in df_all.columns},
          'hover_name': 'person'}

In [ ]:
fig = px.scatter_3d(df_all, x='x', y='y', z='z',
                    color='Redner:innen',
                    opacity=0.7, 
                    size_max=3, 
                    width=800, 
                    height=800, 
                    **params)
fig.show()
fig.write_html('../img/alle-reden-3d-doc-embeddings.html')

## Visualisieren der Reden der TOP 3 Redner

In [ ]:
df_top3 = df_all[(df_all.loc[:, 'person'] == 'Angela Merkel') | \
                 (df_all.loc[:, 'person'] == 'Gerhard Schröder') |\
                 (df_all.loc[:, 'person'] == 'Helmut Kohl')]

In [ ]:
params = {'hover_data': {c: False for c in df_top3.columns},
          'hover_name': 'person'}

In [ ]:
fig = px.scatter_3d(df_top3, x='x', y='y', z='z',
                    color='person',
                    opacity=0.7, 
                    size_max=3, 
                    width=800, 
                    height=800, 
                    **params)
fig.show()
fig.write_html('../img/top3-redner-3d-doc-embeddings.html')

## weitere Optionen

* Labeln nach Parteien und dann visualisieren
* vernünftiges Hover-Template bei Plotly erstellen
* Umsetzung mit Bokeh testen
* Anderer Algorithmus zur Dimensionsreduzierung: t-SNE, PCA
* Reden visualisieren im zeitlichen Verlauf mit Hilfe der Datums-Spalte: 
    * Range-Slider für Zeitraum bei Bokeh ist möglich
    * Was ist bei Plotly dazu möglich